In [ ]:
# !git clone -b master https://github.com/StefanoNasini/Country-by-country-2022-2023-international-relationships
# !cd Country-by-country-2022-2023-international-relationships

In [2]:
import os
root = os.path.dirname(os.getcwd())
import pandas as pd
pd.set_option('display.max_rows', None)

In [3]:
lgbt_full = pd.read_csv(os.path.join(root, "raw_data", "environmental_lgbt", "lgbt_legal_status.csv"), index_col=0)
env_full = pd.read_csv(os.path.join(root, "raw_data", "environmental_lgbt", "environmental.csv"), index_col = 0)
eq_full = pd.read_csv(os.path.join(root, "raw_data", "environmental_lgbt", "world_equality_index.csv"), index_col=0)
mig_full = pd.read_csv(os.path.join(root, "raw_data", "environmental_lgbt", "migration_score.csv"), index_col=0)

# mig2 = pd.read_csv(os.path.join(root, "raw_data", "environmental_lgbt", "migration_nov.csv"))
# mig_top = mig2.iloc[:15, :]
# mig_btm = mig2.iloc[-15:, :]

# mig_countries = list(set(mig2.to_numpy().flatten()))


friends_90tri = pd.read_csv(os.path.join(root, "constructed_network", "final_network", "friends90_tri.csv"), index_col=0)

In [4]:
os.chdir(os.path.join(root, 'src', 'utils'))
from aka import tri2aka, tri2name, name2tri, guess_country
os.chdir(root)

['Y AFGHANISTAN\n', 'A ALBANIA\n', 'Y ALGERIA\n', 'A ANDORRA\n', 'Y ANGOLA\n', 'Y ANTIGUA AND BARBUDA\n', 'Y ARGENTINA\n', 'A ARMENIA\n', 'N AUSTRALIA\n', 'N AUSTRIA\n']


In [5]:
mig_full.index

Index(['new zealand', 'sweden', 'brazil', 'ireland', 'portugal', 'finland',
       'argentina', 'belgium', 'canada', 'luxembourg', 'mexico', 'USA',
       'estonia', 'switzerland', 'norway', 'ukraine', 'australia', 'indonesia',
       'spain', 'germany', 'north macedonia', 'india', 'austria',
       'south africa', 'bulgaria', 'italy', 'hungary', 'korea', 'slovenia',
       'chile', 'denmark', 'albania', 'united kingdom', 'romania', 'iceland',
       'france', 'czechia', 'netherlands', 'israel', 'serbia', 'turkey',
       'malta', 'greece', 'japan', 'moldova', 'cyprus', 'poland', 'lithuania',
       'slovakia', 'united arab emirates', 'russia', 'china', 'croatia',
       'latvia', 'jordan', 'saudi arabia'],
      dtype='object')

# trigramize the index 

In [6]:

# inspect how many countries need manual reassigning

to_rename = {}

for c in list(lgbt_full.index) + list(env_full.index) + list(eq_full.index)+ list(mig_full.index):
    cc = c.replace('-', ' ')
    try:
        tri = name2tri[cc]
    except KeyError:
        try:
            tri = guess_country(cc)
        except Exception:
            tri = None
    to_rename[c] = tri
print(to_rename)

{'bahrain': 'BHR', 'bhutan': 'BTN', 'cambodia': 'KHM', 'china': 'CHN', 'india': 'IND', 'israel': 'ISR', 'japan': 'JPN', 'jordan': 'JOR', 'kazakhstan': 'KAZ', 'kyrgyzstan': 'KGZ', 'laos': 'LAO', 'lebanon': 'LBN', 'mongolia': 'MNG', 'nepal': 'NPL', 'north-korea': 'PRK', 'philippines': 'PHL', 'singapore': 'SGP', 'south-korea': 'KOR', 'syria': 'SYR', 'taiwan': 'TWN', 'tajikistan': 'TJK', 'thailand': 'THA', 'east-timor': 'TLS', 'vietnam': 'VNM', 'albania': 'ALB', 'andorra': 'AND', 'armenia': 'ARM', 'austria': 'AUT', 'azerbaijan': 'AZE', 'belarus': 'BLR', 'belgium': 'BEL', 'bosnia-and-herzegovina': 'BIH', 'bulgaria': 'BGR', 'croatia': 'HRV', 'cyprus': 'CYP', 'czech-republic': 'CZE', 'denmark': 'DNK', 'estonia': 'EST', 'finland': 'FIN', 'france': 'FRA', 'georgia': 'GEO', 'germany': 'DEU', 'greece': 'GRC', 'hungary': 'HUN', 'iceland': 'ISL', 'ireland': 'IRL', 'italy': 'ITA', 'kosovo': 'XKS', 'latvia': 'LVA', 'liechtenstein': 'LIE', 'lithuania': 'LTU', 'luxembourg': 'LUX', 'malta': 'MLT', 'mold

In [7]:
remains = dict((k, v) for (k, v) in to_rename.items() if v==None)
print(remains)

{'cote-divoire': None, 'republic-of-the-congo': None, 'Timor-Leste': None, 'Republic of the Congo': None, 'CÃ´te dâ\x80\x99Ivoire': None, 'USA': None, 'korea': None}


In [8]:
to_rename['cote-divoire'] = "CIV"
to_rename['republic-of-the-congo'] = "COG"
to_rename['Timor-Leste'] = "TLS"
to_rename['Republic of the Congo'] = "COG"
to_rename['CÃ´te dâ\x80\x99Ivoire'] = "CIV"

to_rename['korea'] = "KOR"
to_rename['USA'] = "USA"

to_rename['R‚union'] = guess_country("reunion")
to_rename['China, Hong Kong SAR'] = "HKG"
to_rename['China, Macao SAR'] = "MAC"
to_rename['Cura‡ao'] = "CUW"
to_rename['United States Virgin Islands'] = "VIR"
try:
    mig_full.drop("Channel Islands", axis='index', inplace=True)
except KeyError:
    pass

to_rename["guinea-bissau"] = "GNB"
to_rename["Guinea-Bissau"] = "GNB"

In [9]:
# lgbt = lgbt_full.rename(index=to_rename)
# env = env_full.rename(index=to_rename)
# eq = eq_full.rename(index=to_rename)
# mig = mig_full.rename(index=to_rename)

In [10]:
for c in lgbt_full.index:
    lgbt_full.loc[c, "tri"] = to_rename[c]
lgbt = lgbt_full.set_index("tri")

for c in env_full.index:
    env_full.loc[c, "tri"] = to_rename[c]
env = env_full.set_index("tri")

for c in eq_full.index:
    eq_full.loc[c, "tri"] = to_rename[c]
eq = eq_full.set_index("tri")

for c in mig_full.index:
    mig_full.loc[c, "tri"] = to_rename[c]
mig = mig_full.set_index("tri")

# UN country selection

In [11]:
UN_tri = friends_90tri.columns
lgbt = lgbt.loc[lgbt.index.isin(UN_tri), :]
env = env.loc[env.index.isin(UN_tri), :]
eq = eq.loc[eq.index.isin(UN_tri), :]
# mig = mig.loc[mig.index.isin(UN_tri), :]

In [12]:
# make sure migration contains only UN countries

# Missing data inspection

In [13]:
# The missing data for those countries in adjust growth rate for carbon dioxide
env_full.index[env_full["CDA"].isna()]

Index(['Zambia', 'Tajikistan', 'Bhutan', 'Laos', 'Cote d'Ivoire', 'Mongolia',
       'Ethiopia', 'Mozambique', 'Sri Lanka', 'Rwanda', 'Nepal', 'Cambodia',
       'Madagascar', 'Viet Nam', 'Fiji', 'Myanmar'],
      dtype='object')

In [14]:
# replace missing value with the mean
env["CDA"].fillna(env["CDA"].mean(), inplace=True)

In [15]:
# no missing for lgbt ; no missing for eq


In [16]:
# missing data for migration flow are replaced with median
for tri in UN_tri:
    for item in mig.columns:
        if tri not in mig.index or mig.isna().loc[tri, item]:
            mig.loc[tri, item] = mig[item].median()

In [17]:
mig

,access to nationality,anti discrimination,education,family reunion,health,labour market mobility,permanent residence,political participation
tri,,,,,,,,
NZL,92.0,88.0,76.0,74.0,83.0,59.0,63.0,85.0
SWE,83.0,100.0,93.0,71.0,83.0,91.0,90.0,80.0
BRA,91.0,85.0,14.0,94.0,31.0,67.0,96.0,35.0
IRL,79.0,94.0,45.0,48.0,85.0,22.0,50.0,85.0
PRT,86.0,100.0,69.0,87.0,65.0,94.0,71.0,80.0
FIN,74.0,100.0,88.0,67.0,67.0,91.0,96.0,95.0
ARG,91.0,88.0,26.0,69.0,44.0,44.0,71.0,30.0
BEL,65.0,100.0,74.0,48.0,73.0,56.0,75.0,65.0
CAN,88.0,100.0,86.0,88.0,73.0,76.0,77.0,50.0


In [18]:
# for equality
(eq['Equality Index'] >= 80).sum()

13

# Prerequisite generation

## LGBT 


1. Death penalty for prerequisite 0

2. Legal adoption and Illegal discrimination 

3. Having a long history of legistration of same-sex marriage, at least earlier than 2010. Which involves
c.f. https://en.wikipedia.org/wiki/Timeline_of_same-sex_marriage

By filtering the 2 and 3 criteria, we will get a smaller set of prerequisites-to-1 countries. 

For real choice, we decide upon whether Homosexuality is legal or not.

In [19]:
prerequisite = pd.DataFrame(index= UN_tri, dtype=int)
prerequisite.loc[:, ["LGBT", "environmental", "equality", "migration", "IPalignment"]] = -1

real_choice = pd.DataFrame(index= UN_tri, dtype=int)
real_choice.loc[:, ["LGBT", "environmental", "equality", "migration", "IPalignment"]] = -1

significant_choice = pd.DataFrame(index= UN_tri, dtype=int)
significant_choice.loc[:, ["LGBT", "environmental", "equality", "migration", "IPalignment"]] = -1

significant_choice70 = pd.DataFrame(index= UN_tri, dtype=int)
significant_choice70.loc[:, ["LGBT", "environmental", "equality", "migration", "IPalignment"]] = -1

significant_choice80 = pd.DataFrame(index= UN_tri, dtype=int)
significant_choice80.loc[:, ["LGBT", "environmental", "equality", "migration", "IPalignment"]] = -1

significant_choice85 = pd.DataFrame(index= UN_tri, dtype=int)
significant_choice85.loc[:, ["LGBT", "environmental", "equality", "migration", "IPalignment"]] = -1

In [20]:
real_choice["LGBT"] = 0
for tri in lgbt.index:
    if lgbt.loc[tri, "Homosexuality"] == '✔ Legal':
        real_choice.loc[tri, "LGBT"] = 1

In [21]:
eq_full.sort_values("tri")

,Equality Index,Legal Index,Public Opinion Index,tri
Afghanistan,2,1,2.0,AFG
Angola,62,72,51.0,AGO
Albania,42,72,12.0,ALB
Andorra,82,89,74.0,AND
United Arab Emirates,17,3,32.0,ARE
Argentina,74,86,61.0,ARG
Armenia,27,49,6.0,ARM
Antigua and Barbuda,43,70,17.0,ATG
Australia,79,93,65.0,AUS
Austria,77,91,62.0,AUT


In [22]:
## TODO missing data for LGBT
lgbt = lgbt.sort_index()
lgbt_full = lgbt_full.sort_index()

In [23]:
lgbt_full.loc['libya', :]

Homosexuality                    ✖ Illegal (imprisonment as punishment)
Gay Marriage                                                ✖ Not legal
Censorship                                 ✖ Imprisonment as punishment
Changing Gender                                               ✖ Illegal
Non-binary gender recognition                  ✖ Not legally recognized
Discrimination                                         ✖ No protections
Employment Discrimination                              ✖ No protections
Housing Discrimination                                 ✖ No protections
Adoption                                                      ✖ Illegal
Military                                                      ✖ Illegal
Donating Blood                           ✖ Banned (indefinite deferral)
Conversion Therapy                                         ✖ Not banned
Age of Consent                                                      NaN
tri                                                             

In [24]:
illegals = ["✖ Illegal (up to life in prison as punishment)", "✖ Illegal (death penalty as punishment)", "✖ Illegal (imprisonment as punishment)"]
lgbt_full[lgbt_full["Homosexuality"].isin(illegals)]

,Homosexuality,Gay Marriage,Censorship,Changing Gender,Non-binary gender recognition,Discrimination,Employment Discrimination,Housing Discrimination,Adoption,Military,Donating Blood,Conversion Therapy,Age of Consent,tri
afghanistan,✖ Illegal (death penalty as punishment),✖ Not legal,✖ Other punishment,✖ Illegal,✖ Not legally recognized,✖ No protections,✖ No protections,✖ No protections,✖ Illegal,✖ Illegal,✖ Banned (indefinite deferral),✖ Not banned,NaN,AFG
algeria,✖ Illegal (imprisonment as punishment),✖ Not legal,✖ Imprisonment as punishment,✖ Illegal,Unknown,✖ No protections,✖ No protections,✖ No protections,✖ Single only,✖ Illegal,✖ Banned (indefinite deferral),✖ Not banned,NaN,DZA
bangladesh,✖ Illegal (up to life in prison as punishment),✖ Not legal,✔ No censorship,"✔ Legal, surgery not required",✔ Recognized,✖ No protections,✖ No protections,✖ No protections,✖ Single only,✖ Illegal,Ambiguous,✖ Not banned,NaN,BGD
brunei,✖ Illegal (death penalty as punishment),✖ Not legal,✖ Other punishment,✖ Illegal,✖ Not legally recognized,✖ No protections,✖ No protections,✖ No protections,✖ Illegal,✖ Illegal,✖ Banned (indefinite deferral),✖ Not banned,NaN,BRN
burundi,✖ Illegal (imprisonment as punishment),✖ Not legal,✔ No censorship,Ambiguous,Unknown,✖ Illegal in some contexts,Ambiguous,✖ No protections,✖ Single only,✖ Illegal,✔ Legal,Ambiguous,NaN,BDI
cameroon,✖ Illegal (imprisonment as punishment),✖ Not legal,✖ State-enforced,✖ Illegal,✖ Not legally recognized,✖ No protections,✖ No protections,✖ No protections,✖ Illegal,✖ Illegal,✔ Legal,✖ Not banned,NaN,CMR
chad,✖ Illegal (imprisonment as punishment),✖ Not legal,✔ No censorship,Ambiguous,Unknown,✖ Illegal in some contexts,✖ No protections,✖ No protections,✖ Single only,✖ Illegal,✔ Legal,✖ Not banned,NaN,TCD
comoros,✖ Illegal (imprisonment as punishment),✖ Unrecognized,✔ No censorship,✖ Illegal,Unknown,✖ No protections,✖ No protections,✖ No protections,✖ Single only,✖ Illegal,✖ Banned (indefinite deferral),✖ Not banned,NaN,COM
dominica,✖ Illegal (imprisonment as punishment),✖ Unrecognized,✔ No censorship,Ambiguous,Unknown,✖ Illegal in some contexts,✖ No protections,✖ No protections,✖ Single only,NaN,✔ Legal,Ambiguous,NaN,DMA
eritrea,✖ Illegal (imprisonment as punishment),✖ Not legal,✖ State-enforced,Ambiguous,Unknown,✖ No protections,✖ No protections,✖ No protections,✖ Single only,✖ Illegal,✖ Banned (indefinite deferral),✖ Not banned,NaN,ERI


In [25]:
for tri in lgbt[lgbt["Homosexuality"].isin(illegals)].index:
    prerequisite.loc[tri, "LGBT"] = 0


In [26]:
lgbt1 = lgbt[lgbt["Discrimination"] == "✔ Illegal"]
lgbt2 = lgbt1[lgbt1["Adoption"]== '✔ Legal']
list(sorted(lgbt2.index))
# 25 countries are concretely open to homosexuality in terms of anti-discrimination and adoption. Anchor to 1. 


# Nederlands, spain, canada, belgium, south africa, norway, sweden, iceland

['BEL',
 'BRA',
 'CAN',
 'CHL',
 'COL',
 'CUB',
 'DEU',
 'DNK',
 'ESP',
 'FIN',
 'FRA',
 'HRV',
 'IRL',
 'ISL',
 'ISR',
 'LUX',
 'MLT',
 'NLD',
 'NOR',
 'PRT',
 'SVN',
 'SWE',
 'URY',
 'ZAF']

In [27]:
lgbt3 = map(guess_country, "Netherlands, spain, canada, belgium, south africa, norway, sweden, iceland".split(', '))

In [28]:
lgbt_full

,Homosexuality,Gay Marriage,Censorship,Changing Gender,Non-binary gender recognition,Discrimination,Employment Discrimination,Housing Discrimination,Adoption,Military,Donating Blood,Conversion Therapy,Age of Consent,tri
afghanistan,✖ Illegal (death penalty as punishment),✖ Not legal,✖ Other punishment,✖ Illegal,✖ Not legally recognized,✖ No protections,✖ No protections,✖ No protections,✖ Illegal,✖ Illegal,✖ Banned (indefinite deferral),✖ Not banned,NaN,AFG
albania,✔ Legal,✖ Unrecognized,✔ No censorship,Ambiguous,✖ Not legally recognized,✔ Illegal,✔ Sexual orientation and gender identity,✔ Sexual orientation and gender identity,✖ Single only,"✖ Lesbians, gays, bisexuals permitted, transge...",✔ Legal,✔ Banned,✔ Equal,ALB
algeria,✖ Illegal (imprisonment as punishment),✖ Not legal,✖ Imprisonment as punishment,✖ Illegal,Unknown,✖ No protections,✖ No protections,✖ No protections,✖ Single only,✖ Illegal,✖ Banned (indefinite deferral),✖ Not banned,NaN,DZA
andorra,✔ Legal,✔ Legal,✔ No censorship,"✔ Legal, surgery not required",✖ Not legally recognized,✔ Illegal,✔ Sexual orientation and gender identity,✔ Sexual orientation and gender identity,✔ Legal,NaN,✔ Legal,✖ Not banned,✔ Equal,AND
angola,✔ Legal,✖ Unrecognized,✔ No censorship,"✔ Legal, surgery not required",✖ Not legally recognized,✔ Illegal,✔ Sexual orientation and gender identity,✔ Sexual orientation and gender identity,✖ Single only,"✖ Lesbians, gays, bisexuals permitted, transge...",✔ Legal,✖ Not banned,✔ Equal,AGO
antarctica,✔ Legal,Varies by Region,✔ No censorship,✖ Illegal,✔ Recognized,NaN,NaN,NaN,NaN,NaN,✔ Legal,NaN,✖ Unequal,ATA
antigua-and-barbuda,✔ Legal,✖ Foreign same-sex marriages recognized only,✔ No censorship,✖ Illegal,✖ Not legally recognized,✔ Illegal,✔ Sexual orientation and gender identity,✔ Sexual orientation and gender identity,✖ Single only,✔ Legal,✔ Legal,Ambiguous,✔ Equal,ATG
argentina,✔ Legal,✔ Legal,✔ No censorship,"✔ Legal, surgery not required",✔ Recognized,✖ Illegal in some contexts,Varies by Region,Varies by Region,✔ Legal,"✖ Lesbians, gays, bisexuals permitted, transge...",✔ Legal,Varies by Region,✔ Equal,ARG
armenia,✔ Legal,✖ Not legal,✔ No censorship,"✖ Legal, but requires surgery",✖ Not legally recognized,✖ No protections,✖ No protections,✖ No protections,✖ Single only,"✖ Don't Ask, Don't Tell",✔ Legal,✖ Not banned,✔ Equal,ARM
australia,✔ Legal,✔ Legal,✔ No censorship,"✔ Legal, surgery not required",Varies by Region,✖ Illegal in some contexts,✔ Sexual orientation and gender identity,✔ Sexual orientation and gender identity,✔ Legal,✔ Legal,✖ Banned (3-month deferral),Varies by Region,✔ Equal,AUS


In [29]:
for tri in lgbt.index:
    if (lgbt.loc[tri, "Gay Marriage"] == "✔ Legal") and (lgbt.loc[tri, "Adoption"] == "✔ Legal"):
        prerequisite.loc[tri, "LGBT"] = 1
        print(tri2name[tri])

argentina
australia
austria
belgium
brazil
canada
switzerland
chile
colombia
costa rica
cuba
germany
denmark
spain
finland
france
united kingdom
ireland
iceland
luxembourg
malta
netherlands
norway
new zealand
portugal
slovenia
sweden
uruguay
united states
south africa


In [30]:
for tri in lgbt3:
    prerequisite.loc[tri, "LGBT"] = 1
# tri2name["ZAF"]

## Environmental
1. The adjusted growth rate of carbon dioxide emission (CDA). We select top 15 and bottom 15 as anchor nodes.

In [31]:
pd.set_option('display.max_rows', None)
env["CDA"].sort_values()

tri
IRQ      1.900000
BFA      2.200000
TON      2.300000
MRT      2.400000
PHL      2.800000
BDI      4.100000
LBR      4.200000
HTI      4.400000
MLI      5.400000
NER      5.400000
MDV      7.800000
TLS      8.100000
SDN      8.500000
BEN      8.800000
PAK      8.800000
COM      9.000000
BGD      9.400000
TZA     10.600000
BWA     10.700000
GHA     11.600000
GTM     12.200000
SLE     12.700000
WSM     13.100000
PNG     13.100000
UGA     15.400000
PRY     17.200000
TGO     17.400000
IND     17.600000
KIR     17.900000
GMB     17.900000
GNB     18.100000
SEN     18.200000
TCD     18.300000
NAM     18.700000
KEN     18.700000
DZA     18.800000
KGZ     20.500000
VUT     21.700000
BOL     22.700000
OMN     23.400000
ERI     23.400000
GIN     23.500000
GUY     25.100000
NIC     26.500000
TUR     26.500000
GEO     26.700000
MWI     27.000000
BLZ     27.300000
IDN     27.400000
PER     27.600000
HND     28.100000
IRN     28.200000
ARM     29.200000
ZWE     29.300000
TKM     29.500000
CMR   

In [32]:
import numpy as np

In [33]:
sig40, sig60 = np.percentile(env["CDA"].sort_values(), 40), np.percentile(env["CDA"].sort_values(), 60)
sig30, sig70 = np.percentile(env["CDA"].sort_values(), 30), np.percentile(env["CDA"].sort_values(), 70)
sig20, sig80 = np.percentile(env["CDA"].sort_values(), 20), np.percentile(env["CDA"].sort_values(), 80)
sig15, sig85 = np.percentile(env["CDA"].sort_values(), 15), np.percentile(env["CDA"].sort_values(), 85)


for i, tri in enumerate((env["CDA"].sort_values()).index): # increasing
    if i<10:
        prerequisite.loc[tri, "environmental"] = 0
    if env["CDA"].sort_values()[i] <= sig40:
        significant_choice.loc[tri, "environmental"] = 1
    if env["CDA"].sort_values()[i] <= sig30:
        significant_choice70.loc[tri, "environmental"] = 1
    if env["CDA"].sort_values()[i] <= sig20:
        significant_choice80.loc[tri, "environmental"] = 1
    if env["CDA"].sort_values()[i] <= sig15:
        significant_choice80.loc[tri, "environmental"] = 1
        
for i, tri in enumerate(reversed((env["CDA"].sort_values()).index)):
    if i<10:
        prerequisite.loc[tri, "environmental"] = 1
    if env["CDA"].sort_values(ascending=False)[i] >= sig60:
        significant_choice.loc[tri, "environmental"] = 1
    if env["CDA"].sort_values(ascending=False)[i] >= sig70:
        significant_choice70.loc[tri, "environmental"] = 1
    if env["CDA"].sort_values(ascending=False)[i] >= sig80:
        significant_choice80.loc[tri, "environmental"] = 1
    if env["CDA"].sort_values(ascending=False)[i] >= sig85:
        significant_choice80.loc[tri, "environmental"] = 1
        

        
for i, tri in enumerate((env["CDA"].sort_values()).index):
    if 2*i<len(env.index):
        real_choice.loc[tri, "environmental"] = 0
    else:
        real_choice.loc[tri, "environmental"] = 1


In [34]:
prerequisite.loc[:, "environmental"] == 0

AFG    False
AGO    False
ALB    False
ARE    False
ARG    False
ARM    False
ATG    False
AUS    False
AUT    False
AZE    False
BDI     True
BEL    False
BEN    False
BFA     True
BGD    False
BGR    False
BHR    False
BHS    False
BIH    False
BLR    False
BLZ    False
BOL    False
BRA    False
BRB    False
BRN    False
BTN    False
BWA    False
CAF    False
CAN    False
CHE    False
CHL    False
CHN    False
CIV    False
CMR    False
COD    False
COG    False
COL    False
COM    False
CPV    False
CRI    False
CUB    False
CYP    False
CZE    False
DEU    False
DJI    False
DMA    False
DNK    False
DOM    False
DZA    False
ECU    False
EGY    False
ERI    False
ESP    False
EST    False
ETH    False
FIN    False
FJI    False
FRA    False
GAB    False
GBR    False
GEO    False
GHA    False
GIN    False
GMB    False
GNB    False
GNQ    False
GRC    False
GRD    False
GTM    False
GUY    False
HND    False
HRV    False
HTI     True
HUN    False
IDN    False
IND    False
IRL    False

## Equality
Look at Legal index. choose top 10 and bottom 10. 

In [35]:
lb, ub = eq["Legal Index"].sort_values()[[9, -9]]
siglb, sigub = np.percentile(eq["Legal Index"].sort_values(), 40), np.percentile(eq["Legal Index"].sort_values(), 60)
siglb30, sigub70 = np.percentile(eq["Legal Index"].sort_values(), 30), np.percentile(eq["Legal Index"].sort_values(), 70)
siglb20, sigub80 = np.percentile(eq["Legal Index"].sort_values(), 20), np.percentile(eq["Legal Index"].sort_values(), 80)
siglb15, sigub85 = np.percentile(eq["Legal Index"].sort_values(), 15), np.percentile(eq["Legal Index"].sort_values(), 85)

In [36]:
for i, tri in enumerate(eq.index):
    if eq.loc[tri, "Legal Index"] <= lb:
        prerequisite.loc[tri, "equality"] = 0
    if eq["Legal Index"].sort_values()[i] <= siglb:
        significant_choice.loc[tri, "equality"] = 1
    if eq["Legal Index"].sort_values()[i] <= siglb30:
        significant_choice70.loc[tri, "equality"] = 1
    if eq["Legal Index"].sort_values()[i] <= siglb20:
        significant_choice80.loc[tri, "equality"] = 1
    if eq["Legal Index"].sort_values()[i] <= siglb15:
        significant_choice85.loc[tri, "equality"] = 1

    if eq.loc[tri, "Legal Index"] >= ub:
        prerequisite.loc[tri, "equality"] = 1
    if eq["Legal Index"].sort_values()[i] >= sigub:
        significant_choice.loc[tri, "equality"] = 1
    if eq["Legal Index"].sort_values()[i] >= sigub70:
        significant_choice70.loc[tri, "equality"] = 1
    if eq["Legal Index"].sort_values()[i] >= sigub80:
        significant_choice80.loc[tri, "equality"] = 1
    if eq["Legal Index"].sort_values()[i] >= sigub85:
        significant_choice85.loc[tri, "equality"] = 1


In [37]:
(significant_choice == 1).sum(axis=0)

LGBT               0
environmental    141
equality         151
migration          0
IPalignment        0
dtype: int64

In [38]:
for i, tri in enumerate(eq.index):
    if 2*i < len(eq.index):
        real_choice.loc[tri, "equality"] = 0
    else:
        real_choice.loc[tri, "equality"] = 1

## Migration

For prerequisite 1 : Appear in top for more than four times and appear in bottom less than one time

For prerequisite 0 : Appear in top for less than one time and appear in bottom more than four times


In [39]:
count_pos = dict(zip(UN_tri,[0]*len(UN_tri)) )
count_neg = dict(zip(UN_tri,[0]*len(UN_tri)) )
count_pos60 = dict(zip(UN_tri,[0]*len(UN_tri)) )
count_neg60 = dict(zip(UN_tri,[0]*len(UN_tri)) )
count_pos70 = dict(zip(UN_tri,[0]*len(UN_tri)) )
count_neg70 = dict(zip(UN_tri,[0]*len(UN_tri)) )
count_pos80 = dict(zip(UN_tri,[0]*len(UN_tri)) )
count_neg80 = dict(zip(UN_tri,[0]*len(UN_tri)) )
count_pos85 = dict(zip(UN_tri,[0]*len(UN_tri)) )
count_neg85 = dict(zip(UN_tri,[0]*len(UN_tri)) )

for item in mig.columns:
    for tri in UN_tri:
        if mig.loc[tri, item] >= mig[item].sort_values(ascending=False)[14]:
            count_pos[tri] += 1
        if mig.loc[tri, item] >= np.percentile(mig[item].sort_values(ascending=False), 60):
            count_pos60[tri] += 1
        if mig.loc[tri, item] >= np.percentile(mig[item].sort_values(ascending=False), 70):
            count_pos70[tri] += 1
        if mig.loc[tri, item] >= np.percentile(mig[item].sort_values(ascending=False), 80):
            count_pos80[tri] += 1
        if mig.loc[tri, item] >= np.percentile(mig[item].sort_values(ascending=False), 85):
            count_pos85[tri] += 1
            
        if mig.loc[tri, item] <= mig[item].sort_values(ascending=True)[14]:
            count_neg[tri] += 1
#         if mig.loc[tri, item] <= np.percentile(mig[item].sort_values(ascending=False), 40):
            count_neg60[tri] += 1
#         if mig.loc[tri, item] <= np.percentile(mig[item].sort_values(ascending=False), 30):
            count_neg70[tri] += 1
#         if mig.loc[tri, item] <= np.percentile(mig[item].sort_values(ascending=False), 20):
            count_neg80[tri] += 1
            count_neg85[tri] += 1

In [40]:
for tri in UN_tri:
    if mig.mean(axis=1).sort_values()[tri] >= mig.mean(axis=1).sort_values().mean():
        real_choice.loc[tri, "migration"] = 1
    else:
        real_choice.loc[tri, "migration"] = 0


In [41]:
# for tri in set(count_pos.keys()).union(set(count_neg.keys())):
#     if (tri in count_pos.keys() and count_pos[tri] >= 4) and (tri not in count_neg.keys() or count_neg[tri]<=1):
#         print(f"To prerequisite 1 is country {tri2name[tri]}")
#         prerequisite.loc[tri, 'migration'] = 1

# for tri in set(count_pos.keys()).union(set(count_neg.keys())):
#     if (tri in count_neg.keys() and count_neg[tri] >= 4) and (tri not in count_pos.keys() or count_pos[tri]<=1):
#         print(f"To prerequisite 0 is country {tri2name[tri]}")
#         prerequisite.loc[tri, 'migration'] = 0

In [42]:
guess_country("hungary")
tri = "HUN"
# (tri in count_neg.keys() and count_neg[tri] >= 4) and (tri not in count_pos.keys() or count_pos[tri]<=1)
count_pos["HUN"], count_neg["HUN"]

(2, 3)

In [43]:
# to accomodate Hungary, the criteria needs to be changed to >=3 and <=2

for tri in set(count_pos.keys()).union(set(count_neg.keys())):
    if (tri in count_pos.keys() and count_pos[tri] >= 3) and (tri not in count_neg.keys() or count_neg[tri]<=2):
        print(f"To prerequisite 1 is country {tri2name[tri]}")
        prerequisite.loc[tri, 'migration'] = 1
        significant_choice.loc[tri, 'migration'] = 1
        significant_choice70.loc[tri, 'migration'] = 1
        significant_choice80.loc[tri, 'migration'] = 1
        significant_choice85.loc[tri, 'migration'] = 1


    else:
        if (tri in count_pos60.keys() and count_pos60[tri] >= 3) and (tri not in count_neg60.keys() or count_neg60[tri]<=2):
            print(f"To significant 1 is country {tri2name[tri]}")
            significant_choice.loc[tri, 'migration'] = 1
        if (tri in count_pos70.keys() and count_pos70[tri] >= 3) and (tri not in count_neg70.keys() or count_neg70[tri]<=2):
            print(f"To significant 1 is country {tri2name[tri]}")
            significant_choice70.loc[tri, 'migration'] = 1
        if (tri in count_pos80.keys() and count_pos80[tri] >= 3) and (tri not in count_neg80.keys() or count_neg80[tri]<=2):
            print(f"To significant 1 is country {tri2name[tri]}")
            significant_choice80.loc[tri, 'migration'] = 1
        if (tri in count_pos85.keys() and count_pos85[tri] >= 3) and (tri not in count_neg85.keys() or count_neg85[tri]<=2):
            print(f"To significant 1 is country {tri2name[tri]}")
            significant_choice85.loc[tri, 'migration'] = 1
        
    if (tri in count_neg.keys() and count_neg[tri] >= 3) and (tri not in count_pos.keys() or count_pos[tri]<=2):
        print(f"To prerequisite 0 is country {tri2name[tri]}")
        prerequisite.loc[tri, 'migration'] = 0
        significant_choice.loc[tri, 'migration'] = 0
        significant_choice70.loc[tri, 'migration'] = 1
        significant_choice80.loc[tri, 'migration'] = 1
        significant_choice85.loc[tri, 'migration'] = 1

    else:
        if (tri in count_neg60.keys() and count_neg60[tri] >= 3) and (tri not in count_pos60.keys() or count_pos60[tri]<=2):
            print(f"To significant 0 is country {tri2name[tri]}")
            significant_choice.loc[tri, 'migration'] = 0
        if (tri in count_neg70.keys() and count_neg70[tri] >= 3) and (tri not in count_pos70.keys() or count_pos70[tri]<=2):
            print(f"To significant 0 is country {tri2name[tri]}")
            significant_choice70.loc[tri, 'migration'] = 0
        if (tri in count_neg80.keys() and count_neg80[tri] >= 3) and (tri not in count_pos80.keys() or count_pos80[tri]<=2):
            print(f"To significant 0 is country {tri2name[tri]}")
            significant_choice80.loc[tri, 'migration'] = 0
        if (tri in count_neg85.keys() and count_neg85[tri] >= 3) and (tri not in count_pos85.keys() or count_pos85[tri]<=2):
            print(f"To significant 0 is country {tri2name[tri]}")
            significant_choice85.loc[tri, 'migration'] = 0

To significant 1 is country iran (islamic republic of)
To significant 1 is country iran (islamic republic of)
To significant 1 is country iran (islamic republic of)
To significant 1 is country iran (islamic republic of)
To prerequisite 1 is country argentina
To significant 1 is country costa rica
To significant 1 is country costa rica
To significant 1 is country costa rica
To significant 1 is country costa rica
To significant 1 is country cameroon
To significant 1 is country cameroon
To significant 1 is country cameroon
To significant 1 is country cameroon
To prerequisite 1 is country spain
To prerequisite 0 is country turkiye
To significant 1 is country saint vincent and the grenadines
To significant 1 is country saint vincent and the grenadines
To significant 1 is country saint vincent and the grenadines
To significant 1 is country saint vincent and the grenadines
To prerequisite 0 is country croatia
To significant 1 is country tonga
To significant 1 is country tonga
To significant 1

## Israel-Palestine alignment
To prerequisite 0 are countries who have not recognize Palestine as a state

Israel, united states, Canada, Australia, Austria, Bulgaria, Croatia, Cyprus, 
Czech Republic, Estonia, Greece, Hungary, Latvia, Lithuania, Poland, Romania, Slovakia, Germany, 
Italy, Japan, South Korea, UK

To Prerequisite 1 are countries who don’t recognize Israel as a state:

Afghanistan, Algeria, Bahrain, Bangladesh, Brunei, Comoros, Djibouti, Iran, Iraq, Kuwait, Lebanon, 
Libya, Malaysia, Mauritania, Morocco, North Korea, Oman, Pakistan, Qatar, Saudi Arabia, Somalia, 
Sudan, Syria, Tunisia, United Arab Emirates, Yemen.

In [44]:
IPzero = """Israel, united states, Canada, Australia, Austria, Bulgaria, Croatia, Cyprus, 
Czech Republic, Estonia, Greece, Hungary, Latvia, Lithuania, Poland, Romania, Slovakia, Germany, 
Italy, Japan, South Korea, UK""".split(", ")

IPzero = list(map(guess_country, IPzero))
for tri in IPzero:
    prerequisite.loc[tri, "IPalignment"] = 0

In [45]:
IPone = """Afghanistan, Algeria, Bahrain, Bangladesh, Brunei, Comoros, Djibouti, Iran, Iraq, Kuwait, Lebanon, 
Libya, Malaysia, Mauritania, Morocco, North Korea, Oman, Pakistan, Qatar, Saudi Arabia, Somalia, 
Sudan, Syria, Tunisia, United Arab Emirates, Yemen""".split(", ")

IPone = list(map(guess_country, IPone))
for tri in IPone:
    prerequisite.loc[tri, "IPalignment"] = 1

In [46]:
# for i, tri in enumerate((mig["2015"].sort_values()).index):
#     if i<10:
#         prerequisite.loc[tri, "migration"] = 0
#         print(tri2name[tri])
#     else:
#         break
        
# print()
        
# for i, tri in enumerate(reversed((mig["2015"].sort_values()).index)):
#     if i<10:
#         prerequisite.loc[tri, "migration"] = 1
#         print(tri2name[tri])
#     else:
#         break
        
        
# for i, tri in enumerate((mig["2015"].sort_values()).index):
#     if 2*i<len(mig.index):
#         real_choice.loc[tri, "migration"] = 0
#     else:
#         real_choice.loc[tri, "migration"] = 1

In [47]:
significant_choice

,LGBT,environmental,equality,migration,IPalignment
AFG,-1.0,1.0,1.0,1.0,-1.0
AGO,-1.0,-1.0,1.0,1.0,-1.0
ALB,-1.0,-1.0,-1.0,0.0,-1.0
ARE,-1.0,-1.0,1.0,0.0,-1.0
ARG,-1.0,1.0,1.0,1.0,-1.0
ARM,-1.0,1.0,1.0,1.0,-1.0
ATG,-1.0,-1.0,-1.0,1.0,-1.0
AUS,-1.0,1.0,1.0,1.0,-1.0
AUT,-1.0,1.0,1.0,0.0,-1.0
AZE,-1.0,1.0,1.0,1.0,-1.0


In [48]:
with open(os.path.join(root, 'constructed_network', "predefined.txt"), 'w') as f:
    for i in range(prerequisite.shape[0]):
        for j in range(prerequisite.shape[1]):
            try:
                f.write(str(int(prerequisite.iloc[i, j])))
            except ValueError:
                pass
            f.write(' ')
        f.write("\n")

In [49]:
# import random

# random.seed(0)
# for i in range(prerequisite.shape[0]):
#     for j in range(prerequisite.shape[1]):
#         if random.random() < 0.9:
#             prerequisite.iloc[i, j] = random.randint(0, 1)
#         else:
#             prerequisite.iloc[i, j] = -1

In [60]:
(significant_choice80 == 1).sum()

LGBT               0
environmental     72
equality          76
migration        181
IPalignment        0
dtype: int64

In [50]:
real_choice.isna().sum()

LGBT             0
environmental    0
equality         0
migration        0
IPalignment      0
dtype: int64

In [57]:
real_choice.to_csv(os.path.join(root, "constructed_network", "v_real_choice.csv"))
prerequisite.to_csv(os.path.join(root, "constructed_network", "prerequisite.csv"))
significant_choice.to_csv(os.path.join(root, "constructed_network", "significant_choice.csv"))
significant_choice70.to_csv(os.path.join(root, "constructed_network", "significant_choice70.csv"))
significant_choice80.to_csv(os.path.join(root, "constructed_network", "significant_choice80.csv"))
significant_choice85.to_csv(os.path.join(root, "constructed_network", "significant_choice85.csv"))


In [52]:
(prerequisite==-1).sum()

LGBT             117
environmental    164
equality         165
migration        146
IPalignment      136
dtype: int64

In [53]:
for i, c in enumerate(prerequisite['LGBT']):
    print(f"fixed to {c} is country {i}") if int(c) != -1 else None

fixed to 0.0 is country 0
fixed to 0.0 is country 3
fixed to 1.0 is country 4
fixed to 1.0 is country 7
fixed to 1.0 is country 8
fixed to 0.0 is country 10
fixed to 1.0 is country 11
fixed to 0.0 is country 14
fixed to 1.0 is country 22
fixed to 0.0 is country 24
fixed to 1.0 is country 28
fixed to 1.0 is country 29
fixed to 1.0 is country 30
fixed to 0.0 is country 33
fixed to 1.0 is country 36
fixed to 0.0 is country 37
fixed to 1.0 is country 39
fixed to 1.0 is country 40
fixed to 1.0 is country 43
fixed to 0.0 is country 45
fixed to 1.0 is country 46
fixed to 0.0 is country 48
fixed to 0.0 is country 51
fixed to 1.0 is country 52
fixed to 0.0 is country 54
fixed to 1.0 is country 55
fixed to 1.0 is country 57
fixed to 1.0 is country 59
fixed to 0.0 is country 62
fixed to 0.0 is country 63
fixed to 1.0 is country 76
fixed to 0.0 is country 77
fixed to 1.0 is country 79
fixed to 0.0 is country 86
fixed to 0.0 is country 95
fixed to 0.0 is country 96
fixed to 0.0 is country 98
fixed 

In [56]:
real_choice['LGBT'].mean()

0.6739130434782609